## 尝试1

### 爬取A股上市公司年报链接并存入Excel

In [11]:
import requests
import re
import openpyxl
import time

# 设置搜索参数
search_key = "年报"
url_template = "http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey={}&sdate=2019-01-01&edate=2020-01-01&isfulltext=false&sortName=pubdate&sortType=desc&pageNum={}&type=shj"
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'}


In [12]:
# 发送HTTP请求并获取响应
all_results = []
page_num = 1   # 设置首页
total_pages = 1   # 初始化最大页面
max_retries = 2    # 最大重试次数
retry_count = 0

while page_num <= total_pages:
    url = url_template.format(search_key, page_num)
    response = None

    # 重试机制
    while retry_count <= max_retries:
        # 发送请求
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            break
        except requests.exceptions.RequestException as e:
            print(f"出现错误！ ： {e}")
            print(f"5秒后重试...")
            time.sleep(5)
            retry_count += 1
    
    if retry_count > max_retries:
        print(f"{max_retries} 次重试后均失败，跳过第{page_num}页。")
        page_num += 1
        retry_count = 0
        continue

    # 解析数据
    try:
        data = response.json()
        all_results.extend(data["announcements"])
        total_pages = data["totalpages"]
        print(f"正在下载第 {page_num}/{total_pages} 页")
        page_num += 1
        retry_count = 0
    except (ValueError, KeyError) as e:
        print(f"Error parsing response data: {e}")
        print(f"5秒后重试...")
        time.sleep(5)
        retry_count += 1
        continue

正在下载第 1/1611 页
正在下载第 2/1611 页
正在下载第 3/1611 页
正在下载第 4/1611 页
正在下载第 5/1611 页
正在下载第 6/1611 页
正在下载第 7/1611 页
正在下载第 8/1611 页
正在下载第 9/1611 页
正在下载第 10/1611 页
正在下载第 11/1611 页
正在下载第 12/1611 页
正在下载第 13/1611 页
正在下载第 14/1611 页
正在下载第 15/1611 页
正在下载第 16/1611 页
正在下载第 17/1611 页
正在下载第 18/1611 页
正在下载第 19/1611 页
正在下载第 20/1611 页
正在下载第 21/1611 页
正在下载第 22/1611 页
正在下载第 23/1611 页
正在下载第 24/1611 页
正在下载第 25/1611 页
正在下载第 26/1611 页
正在下载第 27/1611 页
正在下载第 28/1611 页
正在下载第 29/1611 页
正在下载第 30/1611 页
正在下载第 31/1611 页
正在下载第 32/1611 页
正在下载第 33/1611 页
正在下载第 34/1611 页
正在下载第 35/1611 页
正在下载第 36/1611 页
正在下载第 37/1611 页
正在下载第 38/1611 页
正在下载第 39/1611 页
正在下载第 40/1611 页
正在下载第 41/1611 页
正在下载第 42/1611 页
正在下载第 43/1611 页
正在下载第 44/1611 页
正在下载第 45/1611 页
正在下载第 46/1611 页
正在下载第 47/1611 页
正在下载第 48/1611 页
正在下载第 49/1611 页
正在下载第 50/1611 页
正在下载第 51/1611 页
正在下载第 52/1611 页
正在下载第 53/1611 页
正在下载第 54/1611 页
正在下载第 55/1611 页
正在下载第 56/1611 页
正在下载第 57/1611 页
正在下载第 58/1611 页
正在下载第 59/1611 页
正在下载第 60/1610 页
正在下载第 61/1611 页
正在下载第 62/1611 页
正在下载第 63/1611 页
正

In [13]:
# 创建 Excel 文件并添加表头
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = "Search Results"
worksheet.append(["公司代码", "公司简称", "标题", "发布日期", "年报链接"])

# 定义需要删除的标题关键词
exclude_keywords = ["摘要", "英文版", "披露", "风险提示", "督导","反馈","回复","业绩","说明会","意见","审核","独立董事",
                    "半年","已取消","补充","提示性","制度","规程","审计","ST","公告","声明","说明","受托"]

# 解析搜索结果并添加到 Excel 表格中
for item in all_results:
    company_code = item["secCode"]
    company_name = item["secName"]
    title = item["announcementTitle"].strip()

    # 剔除不需要的样式和特殊符号，并重新组合标题
    title = re.sub(r"<.*?>", "", title)
    title = title.replace("：", "")
    title = f"《{title}》"

    adjunct_url = item["adjunctUrl"]
    year = re.search(r"\d{4}", adjunct_url).group()
    publish_time = f"{year}"
    # 拼串，将PDF文件的下载链接拼完整！
    announcement_url = f"http://static.cninfo.com.cn/{adjunct_url}"

    # 检查标题是否包含排除关键词
    exclude_flag = False
    for keyword in exclude_keywords:
        if keyword in title:
            exclude_flag = True
            break

    # 如果标题不包含排除关键词，则将搜索结果添加到Excel表格中
    if not exclude_flag:
        worksheet.append([company_code, company_name, title, publish_time, announcement_url])

In [14]:
workbook.save(f"上市公司年报链接_2018.xlsx")

## 尝试2

### 1.A股上市公司年报链接获取

In [15]:
#首先引入第三方库
import requests
import re
import openpyxl
import time

#定义一个访问接口的函数
def get_report(page_num, date):
    url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
        "Content-Length": "195",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Host": "www.cninfo.com.cn",
        "Origin": "http://www.cninfo.com.cn",
        "Proxy-Connection": "keep-alive",
        "Referer": "http://www.cninfo.com.cn/new/commonUrl/pageOfSearch?url=disclosure/list/search&checkedCategory=category_ndbg_szsh",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.42",
        "X-Requested-With": "XMLHttpRequest"
    }

    '''
    参数信息
     plate: sz;sh, 表示沪深两市
     seDate：查询时间
    '''
    data = {
        "pageNum": page_num,
        "pageSize": 30,
        "column": "szse",
        "tabName": "fulltext",
        "plate": "sz;sh",
        "searchkey": "",
        "secid": "",
        "category": "category_ndbg_szsh",
        "trade": "",
        "seDate": date,
        "sortName": "code",
        "sortType": "asc",
        "isHLtitle": "false"
    }

    response = requests.post(url, data=data, headers=headers)
    return response


`get_report`函数包含两个参数page_num和date，我们可以控制当前页码来遍历整个json文件，并用date来控制需要查询的时间范围。

In [16]:
def download_report(date):
    global counter
    all_results = []
    page_num = 1
    response_test = get_report(page_num, date)
    data_test = response_test.json()
    total_pages = data_test["totalpages"]
    max_retries = 3  #最大重试次数
    retry_count = 0  #当前重试次数
    while page_num <= total_pages:
        response = None

        # 重试机制
        while retry_count <= max_retries:
            # 发送请求
            try:
                # response = requests.post(url, data=data,headers=headers)
                response = get_report(page_num,date)
                response.raise_for_status()
                break
            except requests.exceptions.RequestException as e:
                print(f"出现错误！: {e}")
                print(f"5秒后重试...")
                time.sleep(5)
                retry_count += 1

        if retry_count > max_retries:
            print(f"{max_retries} 次重试后均失败. 跳过第 {page_num}页.")
            page_num += 1
            retry_count = 0
            continue
        else:
            # 解析数据
            try:
                data = response.json()
                # print(f"正在下载第 {page_num}/{total_pages} 页")
                print(f"\r正在下载第 {counter}/{sum} 页", end='')
                # 尝试解析公告数据，如果解析失败则重试
                retry_count = 0
                while True:
                    try:
                        if data["announcements"] is None:
                            raise Exception("公告数据为空")
                        else:
                            all_results.extend(data["announcements"])
                        break
                    except (TypeError, KeyError) as e:
                        print(f"解析公告数据失败: {e}")
                        print(f"5秒后重试...")
                        time.sleep(5)
                        retry_count += 1
                        if retry_count > max_retries:
                            raise Exception("达到最大重试次数，跳过此页")
                        continue
                page_num += 1
                counter += 1
            except (ValueError, KeyError) as e:
                print(f"解析响应数据失败: {e}")
                print(f"5秒后重试...")
                time.sleep(5)
                retry_count += 1
                if retry_count > max_retries:
                    raise Exception("达到最大重试次数，跳过此页")
                continue
    return all_results

In [17]:
time_segments = [
    f"{year}-01-01~{year}-04-01",
    f"{year}-04-02~{year}-04-15",
    f"{year}-04-16~{year}-04-22",
    f"{year}-04-23~{year}-04-26",
    f"{year}-04-27~{year}-04-28",
    f"{year}-04-29~{year}-04-30"
]

In [18]:
def main(year):
    # 计数器
    global sum
    date_count = f"{year}-01-01~{year}-04-30"
    response = get_report(1, date_count)
    data = response.json()
    sum = data["totalpages"]
    year = year + 1
    all_results = []
    time_segments = [
        f"{year}-01-01~{year}-04-01",
        f"{year}-04-02~{year}-04-15",
        f"{year}-04-16~{year}-04-22",
        f"{year}-04-23~{year}-04-26",
        f"{year}-04-27~{year}-04-28",
        f"{year}-04-29~{year}-04-30"
    ]
    for i in time_segments:
        results = download_report(i)
        all_results.extend(results)

    
    # 创建Excel文件并添加表头
    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    worksheet.title = "年报链接"
    worksheet.append(["公司代码", "公司简称", "标题", "年份", "年报链接"])

    # 解析搜索结果并添加到Excel表格中
    for item in all_results:
        company_code = item["secCode"]
        company_name = item["secName"]
        title = item["announcementTitle"].strip()
        # 剔除不需要的样式和特殊符号，并重新组合标题
        title = re.sub(r"<.*?>", "", title)
        title = title.replace("：", "")
        title = f" 《{title}》 "

        adjunct_url = item["adjunctUrl"]
        year = re.search(r"\d{4}", title)
        if year:
            year = year.group()
        else:
            year = setYear
        time = f"{year}"
        announcement_url = f"http://static.cninfo.com.cn/{adjunct_url}"

        # 检查标题是否包含排除关键词
        exclude_flag = False
        for keyword in exclude_keywords:
            if keyword in title:
                exclude_flag = True
                break
        
        # 如果标题不包含排除关键词，则将搜索结果添加到Excel表格中
        if not exclude_flag:
            worksheet.append([company_code, company_name, title, time, announcement_url])
    workbook.save(f"年报链接_{setYear}.xlsx")

In [22]:
if __name__ == '__main__':
    # 全局变量
    # 排除列表可以加入'更正后','修订版'来规避数据重复或公司发布之前年份的年报修订版等问题
    exclude_keywords = ['英文', '摘要','已取消','公告']
    global counter
    global sum
    counter = 1  # 计数器
    setYear = 2022  #设置下载年份

    #for setYear in range(2007,2022):
    main(setYear)
    #     print(f"{setYear}年年报下载完成")

正在下载第 338/324 页

### 2.多线程批量下载+多线程PDF转TXT

In [25]:
import requests
import pandas as pd
import os
import multiprocessing
import pdfplumber
import logging
import re

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# 读取Excel文件
try:
    df = pd.read_excel('年报链接_2015.xlsx')
except Exception as e:
    logging.error(f"读取失败！！ {e}")
    #return

In [26]:
# 读取文件内容并存储为字典
content_dict = ((row['公司代码'], row['公司简称'], row['年份'], row['年报链接']) for _,row in df.iterrows())
# 我们分别用code, name, year, pdf_url四个变量来接受这些值

In [27]:
# 创建存储文件的文件夹
pdf_dir = 'pdf年报'
txt_dir = 'txt年报'
try:
    os.makedirs(pdf_dir, exist_ok=True)
    os.makedirs(txt_dir, exist_ok=True)
except Exception as e:
    logging.error(f"创建文件夹失败！请检查权限！ {e}")
    #return

多线程操作

In [32]:
#开启多线程
with multiprocessing.Pool() as pool:
    for code,name,year,pdf_url in content_dict:
        txt_file_name = f"{code}_{name}_{year}.txt"
        txt_file_path = os.path.join(txt_dir, txt_file_name)
        # 检测文件是否已经存在
        if os.path.exists(txt_file_path):
            logging.info(f"{txt_file_name} 已存在，跳过.")
        else:
            pool.apply_async(convert, args=(code, name, year, pdf_url, pdf_dir, txt_dir))

    pool.close()
    pool.join()

下载和转换

In [31]:
def convert(code, name, year, pdf_url, pdf_dir, txt_dir):
    pdf_file_path = os.path.join(pdf_dir, f"{code}_{name}_{year}.pdf")
    txt_file_path = os.path.join(txt_dir, re.sub(r'[\\/:*?"<>|]', '', f"{code}_{name}_{year}.txt"))

    try:
        # 下载PDF文件
        if not os.path.exists(pdf_file_path):
            with requests.get(pdf_url, stream=True) as r:
                r.raise_for_status()
                with open(pdf_file_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
        # 转换PDF文件为TXT文件
        with pdfplumber.open(pdf_file_path) as pdf:
            with open(txt_file_path, 'w', encoding='utf-8') as f:
                for page in pdf.pages:
                    text = page.extract_text()
                    f.write(text)
        
        logging.info(f"{txt_file_path}")

    except Exception as e:
        logging.error(f"出错了- {code}_{name}_{year}. {e}")

    else:
        # 删除已转换的PDF文件，以节省空间
        os.remove(pdf_file_path)
        logging.info(f"{pdf_file_path} 已被删除。")

### 3.提取MD&A部分

In [27]:
import pdfplumber
txt_file_path = "txt年报/1.txt"

with pdfplumber.open('深圳能源：2021年年度报告.PDF') as pdf:
    start_page = 0
    end_page = 0
    for i in range(10):  # 循环前10页，找到目录页
        content = pdf.pages[i].extract_text()
        if "......." in content:
            content = content.split('\n')
            #print(content)
            break
     
    for j in range(len(content)):
        if "与分析" in content[j]:
            start_page = int(content[j][-2:])
            end_page = int(content[j+1][-3:])
            break
    
    with open(txt_file_path, 'w', encoding='utf-8') as f:
        for page in range(start_page-1,end_page-1):
            text = pdf.pages[page].extract_text()
            f.write(text)
    
    with open(txt_file_path, 'w', encoding='utf-8') as f:
        print(f)

In [35]:
text = []
with open(txt_file_path, 'r', encoding='utf-8') as f:
    text = f.readlines()
    for line_num in range(len(text)):
        if len(text[line_num]) > 40:
            text[line_num] = text[line_num].replace('\n', '')
with open(txt_file_path, 'w', encoding='utf-8') as f:
    f.writelines(text)

## 爬虫基础

In [8]:
import requests
from bs4 import BeautifulSoup

In [12]:
head = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get("http://books.toscrape.com/", headers=head)
'''
if response.ok:
    print(response.text)
else:
    print("请求失败")
'''
content = response.text
soup = BeautifulSoup(content, "html.parser")

all_prices = soup.findAll("p", attrs={"class": "price_color"})
for price in all_prices:
    print(price.string)

all_titles = soup.findAll("h3")
for title in all_titles:
    all_links = title.findAll("a")
    for link in all_links:
        print(link.string)

Â£51.77
Â£53.74
Â£50.10
Â£47.82
Â£54.23
Â£22.65
Â£33.34
Â£17.93
Â£22.60
Â£52.15
Â£13.99
Â£20.66
Â£17.46
Â£52.29
Â£35.02
Â£57.25
Â£23.88
Â£37.59
Â£51.33
Â£45.17
A Light in the ...
Tipping the Velvet
Soumission
Sharp Objects
Sapiens: A Brief History ...
The Requiem Red
The Dirty Little Secrets ...
The Coming Woman: A ...
The Boys in the ...
The Black Maria
Starving Hearts (Triangular Trade ...
Shakespeare's Sonnets
Set Me Free
Scott Pilgrim's Precious Little ...
Rip it Up and ...
Our Band Could Be ...
Olio
Mesaerion: The Best Science ...
Libertarianism for Beginners
It's Only the Himalayas


## 使用 Selenium 框架

In [6]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service

# 创建 WebDriver 对象，指明使用edge浏览器驱动
wd = webdriver.Edge(service=Service(r'C:\Users\songqiang\Downloads\msedgedriver.exe'))

# 调用 WebDriver 对象的get方法，可以让浏览器打开指定网址
wd.get('https://www.bing.com')